# Calculate Simplified Ikeda and insert into DB

In [ ]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 15, 5

from rolldecay.simplified_ikeda import calculate_roll_damping
from rolldecay import equations
import sympy as sp
from rolldecay import symbols
from rolldecayestimators.substitute_dynamic_symbols import lambdify

import data
import copy
from rolldecay import database

In [ ]:
df_rolldecay = database.load()
df_rolldecay.fillna(0, inplace=True)

In [ ]:
def calculate(row, PHI):
    
    LPP = row.lpp
    Beam = row.beam
    DRAFT = (row.TA + row.TF)/2
    
    lBK = row.BKL
    bBK = row.BKB
    OMEGA = row.omega0
    OG = (-row.kg+DRAFT)
    CB = row.Volume/(row.lpp*row.beam*DRAFT)
    CMID = row.A0
    
    B44HAT, BFHAT, BWHAT, BEHAT, BBKHAT = calculate_roll_damping(LPP,Beam,CB,CMID,OG,PHI,lBK,bBK,OMEGA,DRAFT)
    s = pd.Series()
    s['B44HAT'] = B44HAT
    s['BFHAT'] = BFHAT
    s['BWHAT'] = BWHAT
    s['BEHAT'] = BEHAT
    s['BBKHAT'] = BBKHAT
    return s
    

In [ ]:
required = ['lpp','beam','TA','TF','BKL','BKB','omega0','kg','Volume','A0']

In [ ]:
df_rolldecay.dropna(subset=required, inplace=True)

In [ ]:
mask = ((df_rolldecay['ship_speed']==0) &  ## Zero speed!
        (df_rolldecay[required] > 0).all(axis=1)
       )
df = df_rolldecay.loc[mask].copy()

In [ ]:
fig,axess = plt.subplots(nrows=4,ncols=3)
fig.set_size_inches(15,15)
for r,ax in zip(required, axess.flatten()):
    df[r].hist(bins=20, ax=ax)
    ax.set_title(r)

In [ ]:
result = df.apply(func=calculate, PHI=1, axis=1)
df = pd.merge(left=df,right=result,left_index=True, right_index=True)

In [ ]:
df.hist('B44HAT', bins=50)

In [ ]:
mask = ((df['B44HAT'] > df['B44HAT'].quantile(0)) &
        (df['B44HAT'] < df['B44HAT'].quantile(0.90)) 
       )
df2 = df.loc[mask].copy()

In [ ]:
df2.hist('B44HAT', bins=20)

In [ ]:
equations.zeta_equation

In [ ]:
sp.solve(equations.zeta_equation, symbols.B_1)[0]

In [ ]:
omega0_equation_linear = equations.omega0_equation.subs(symbols.C,sp.solve(equations.C_equation_linear,symbols.C)[0])
omega0_equation_linear

In [ ]:
A44 = sp.solve(omega0_equation_linear, symbols.A_44)[0]
eq = equations.zeta_equation.subs(symbols.A_44,A44)
eq

In [ ]:
B_1_expression = B44_expression = sp.solve(eq,symbols.B_1)[0]  # I guess that B_1 and B44 is the same thing for linear...

In [ ]:
B44_expression

In [ ]:
B44_lambda = lambdify(B44_expression)

In [ ]:
rho = 1000
g=9.81
Volume = df2['Volume']
beam = df2['beam']
m = rho*Volume
GM = df2['gm'].values
zeta = df2['zeta']
omega0 = df2['omega0']

In [ ]:
df2['B44_db'] = B44 = B44_lambda(GM=GM, m=m, omega0=omega0, zeta=zeta, g=g)

In [ ]:
df2['B44_hat_db'] = B44/(rho*Volume*beam**2)*np.sqrt(beam/(2*g))

In [ ]:
mask = ((df2['B44_hat_db'] > df2['B44_hat_db'].quantile(0)) &
        (df2['B44_hat_db'] < df2['B44_hat_db'].quantile(0.95)) 
       )
df2 = df2.loc[mask].copy()

In [ ]:
fig,ax = plt.subplots()
bins = np.linspace(0,df2[['B44HAT','B44_hat_db']].max().max(),50)
df2.hist('B44HAT', label='Ikeda', bins=bins, ax=ax)
df2.hist('B44_hat_db',label='Roll damping DB', bins=bins, ax=ax, alpha=0.75)

ax.legend()

In [ ]:
fig,ax=plt.subplots()
df2.plot(x='B44_hat_db', y='B44HAT', style='o', alpha=0.3, ax=ax)
ax.axis('equal');

In [ ]:
import seaborn as sns

sns.regplot(x='B44_hat_db', y='B44HAT', data=df2);